### Setup: Acessando Inputs
* df_pais: database original da BD desatualizada
* df_iso: database com códigos iso

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

path = os.path.abspath(os.path.join('..', 'input'))
path_data_base = os.path.join(path, 'pais_old.csv')
df_pais = pd.read_csv(path_data_base)

In [ ]:
path_data_iso = os.path.join(path, 'iso.csv')
df_iso = pd.read_csv(path_data_iso, na_filter=False)
df_iso = df_iso.drop(['Numeric'], axis=1)

### Tratamento e Output

In [ ]:
df = df_pais.merge(df_iso, how='outer', left_on='sigla_iso3',
                   right_on='Alpha-3 code')
df['nome_en'] = df['English short name']
df['sigla_iso2'] = df['Alpha-2 code']
df['sigla_iso3'] = df['Alpha-3 code']
df = df.drop(['Alpha-2 code', 'Alpha-3 code', 'English short name'], axis=1)
df['nome_en'] = df['nome_en'].str.replace("*", "", regex=False)

In [ ]:
#adicionando continentes
path_data_owd = os.path.join(path, 'ourworldindata.csv')
df_owd = pd.read_csv(path_data_owd, na_filter=False)
df_owd.drop(columns=['Entity', 'Year'], inplace=True)

In [ ]:
#merge entre as duas bases
df = df.merge(df_owd, how='inner', left_on='sigla_iso3',
              right_on='Code')
df_owd.drop(columns=['Code'], inplace=True)

In [ ]:
#código no padrão BD
path_data_continente = os.path.join(path, 'continente.csv')
df_continente = pd.read_csv(path_data_continente, na_filter=False)

In [ ]:
df = df.merge(df_continente, how='inner', left_on='Continent',
              right_on='nome_en', suffixes=('', '_cont'))
df.sigla_continente = df.sigla
df.drop(columns=['Code', 'Continent', 'nome_pt_cont',
        'nome_en_cont', 'sigla'], inplace=True)

In [ ]:
#adicionando códigos que faltam
path_data_codes = os.path.join(path, 'fao.xlsx')
df_codes = pd.read_excel(path_data_codes, sheet_name='Codes', na_filter=False)
df_english = pd.read_excel(
    path_data_codes, sheet_name='English', na_filter=False)

In [ ]:
#merge entre bases da FAO
df_code_missing = df_codes.merge(df_english, on='ID')
df_code_missing = df_code_missing[[
    'ISO3', 'FAOSTAT', 'GAULCODE', 'UNDP', 'ADJECTIVE/INHABITANT', 'FULL NAME', 'M49']]

In [ ]:
#merge com df
df = df.merge(df_code_missing, how='inner', left_on='sigla_iso3',
              right_on='ISO3')
df['id_pais_fao'] = df['FAOSTAT']
df['id_pais_gaul'] = df['GAULCODE']
df['sigla_pnud'] = df['UNDP']
df['nome_oficial_en'] = df['FULL NAME']
df['nacionalidade_en'] = df['ADJECTIVE/INHABITANT']
df['id_pais_ocde'] = df['M49']
df.rename(columns={'id_pais_ocde': 'id_pais_m49'}, inplace=True)
df = df.drop(['FAOSTAT', 'GAULCODE', 'UNDP', 'FULL NAME',
             'ADJECTIVE/INHABITANT', 'ISO3', 'M49'], axis=1)

In [ ]:
#transformar coluna em string
def change_format_str(item):
    my_formatter = "{0:0>2d}"
    new = int(item)
    my_formatter.format(new)
    return str(new)
df['id_pais_m49'] = df['id_pais_m49'].apply(lambda x: change_format_str(x))

In [ ]:
#nome_pt
nome_pt_dict = {'Gibraltar': 'Gibraltar',
                'Guernsey': 'Guernsey',
                'Holy See (the)': 'Vaticano',
                'Isle of Man': 'Ilha de Man',
                'Jersey': 'Jersey',
                'Liechtenstein': 'Liechtenstein',
                'Svalbard and Jan Mayen': 'Svalbard e Jan Mayen',
                'Åland Islands': 'Ilhas Åland',
                'Mayotte': 'Mayotte',
                'Réunion': 'Ilha da Reunião',
                'Saint Helena, Ascension and Tristan da Cunha': 'Santa Helena, Ascensão e Tristão da Cunha',
                'Western Sahara': 'Saara Ocidental',
                'Anguilla': 'Anguila',
                'Aruba': 'Aruba',
                'Bermuda': 'Bermudas',
                'Bonaire, Sint Eustatius and Saba': 'Países Baixos Caribenhos',
                'Cayman Islands (the)': 'Ilhas Cayman',
                'Curaçao': 'Curaçao',
                'Greenland': 'Groenlândia',
                'Guadeloupe': 'Guadalupe',
                'Martinique': 'Martinica',
                'Montserrat': 'Monserrate',
                'Puerto Rico': 'Porto Rico',
                'Saint Barthélemy': 'São Bartolomeu',
                'Saint Martin (French part)': 'São Martinho (França)',
                'Saint Pierre and Miquelon': 'São Pedro e Miquelon',
                'Sint Maarten (Dutch part)': 'São Martinho (Países Baixos)',
                'Turks and Caicos Islands (the)': 'Ilhas Turcas e Caicos',
                'Virgin Islands (British)': 'Ilhas Virgens Britânicas',
                'Virgin Islands (U.S.)': 'Ilhas Virgens Americanas',
                'Falkland Islands (the) [Malvinas]': 'Ilhas Falkland (Ilhas Malvinas)',
                'French Guiana': 'Guiana Francesa',
                'South Georgia and the South Sandwich Islands': 'Ilhas Geórgia do Sul e Sandwich do Sul',
                'Afghanistan': 'Afeganistão',
                'British Indian Ocean Territory (the)': 'Território do Oceano Índico Britânico',
                'Christmas Island': 'Ilha Christmas',
                'Cocos (Keeling) Islands (the)': 'Ilhas Cocos',
                'Hong Kong': 'Hong Kong',
                'Macao': 'Macau',
                'Palestine, State of': 'Palestina',
                'Taiwan (Province of China)': 'Taiwan',
                'American Samoa': 'Samoa Americana',
                'French Polynesia': 'Polinésia Francesa',
                'Guam': 'Guam',
                'New Caledonia': 'Nova Caledônia',
                'Norfolk Island': 'Ilha Norfolk',
                'Northern Mariana Islands (the)': 'Ilhas Marianas do Norte',
                'Pitcairn': 'Pitcairn',
                'United States Minor Outlying Islands (the)': 'Ilhas Menores Distantes dos Estados Unidos',
                'Wallis and Futuna': 'Wallis e Futuna',
                'Antarctica': 'Antártica',
                'Bouvet Island': 'Ilha Bouvet',
                'French Southern Territories (the)': 'Territórios Franceses do Sul',
                'Heard Island and McDonald Islands': 'Ilha Heard e Ilhas McDonald'}


In [ ]:
df.nome_pt = df.nome_pt.fillna(df.nome_en.map(nome_pt_dict))

In [ ]:
#nacionalidade
nacionalidade_dict = {'Ilhas Feroe': 'Feroesa',
                      'Gibraltar': 'Gibraltar; de Gibraltar',
                      'Guernsey': 'Guernesiana',
                      'Vaticano': 'da Santa Sé',
                      'Ilha de Man': 'Manesa',
                      'Jersey': 'Jersiana',
                      'Liechtenstein': 'Liechtensteinense',
                      'Ilhas Åland': 'Alandesa',
                      'Seicheles': 'Seichelense',
                      'Sudão Do Sul': 'Sul-Sudanesa',
                      'Mayotte': 'Maiotense',
                      'Ilha da Reunião': 'Reunionense',
                      'Santa Helena, Ascensão e Tristão da Cunha': 'de Santa Helena, Ascensão e Tristão da Cunha',
                      'Saara Ocidental': 'Saaráui',
                      'São Cristóvão e Nevis': 'de São Cristóvão e Nevis',
                      'São Vicente e Granadinas': 'de São Vicente e Granadinas',
                      'Anguila': 'Anguilana',
                      'Aruba': 'Arubana',
                      'Bermudas': 'Bermudense',
                      'Países Baixos Caribenhos': 'de Bonaire, Santo Eustáquio e Saba',
                      'Ilhas Cayman': 'Caimanesa',
                      'Curaçao': 'Curaçauense',
                      'Groenlândia': 'Groenlandesa',
                      'Guadalupe': 'Guadalupense',
                      'Martinica': 'Martinicana',
                      'Monserrate': 'Monserratense',
                      'Porto Rico': 'Porto-Riquenho',
                      'São Bartolomeu': 'São-Bartolomeense',
                      'São Martinho (França)': 'São-Martinhense (França)',
                      'São Pedro e Miquelon': 'de São Pedro e Miquelon',
                      'São Martinho (Países Baixos)': 'São-Martinhense (Países Baixos)',
                      'Ilhas Turcas e Caicos': 'das Ilhas Turcas e Caicos',
                      'Ilhas Virgens Britânicas': 'Virginense (Britânica)',
                      'Ilhas Virgens Americanas': 'Virginense (Americana)',
                      'Ilhas Falkland (Ilhas Malvinas)': 'Malvinense',
                      'Guiana Francesa': 'da Guiana Francesa',
                      'Ilhas Geórgia do Sul e Sandwich do Sul': 'Sul-Georgiana',
                      'Afeganistão': 'Afegã',
                      'Ilha Christmas': 'da Ilha Christmas',
                      'Ilhas Cocos': 'das Ilhas Cocos',
                      'Hong Kong': 'Honconguesa',
                      'Macau': 'Macaense',
                      'Palestina': 'Palestina',
                      'Taiwan': 'Taiwanesa',
                      'Tokelau': 'de Tokelau',
                      'Samoa Americana': 'Samoana',
                      'Polinésia Francesa': 'Polinésia',
                      'Guam': 'Guamesa',
                      'Nova Caledônia': 'Neocaledônia',
                      'Ilha Norfolk': 'Norfolquina',
                      'Pitcairn': 'Pitcairnesa',
                      'Ilhas Marianas do Norte': 'Norte-Marianense',
                      'Wallis e Futuna': 'de Wallis e Futuna',
                      'Antártica': 'Antártica',
                      'Territórios Franceses do Sul': 'dos Territórios Franceses do Sul'}


In [ ]:
df.nacionalidade = df.nacionalidade.fillna(df.nome_pt.map(nacionalidade_dict))
df.drop(columns=['nacionalidade_en'], inplace=True)

Mais alguns tratamentos finais e output

In [ ]:
df = df.replace(r'^\s*$', np.NaN, regex=True)

In [ ]:
df = df.rename(columns={'sigla_iso3': 'sigla_pais_iso3', 'sigla_iso2': 'sigla_pais_iso2',
                        'sigla_pnud': 'sigla_pais_pnud', 'nome_pt': 'nome', 'nome_en': 'nome_ingles', 
                        'nome_oficial_en': 'nome_oficial_ingles'})


In [ ]:
path_out = os.path.abspath(os.path.join('..', 'output'))
path_data_out = os.path.join(path_out, 'pais.csv')
if (not os.path.isdir(path_out)):
    os.mkdir(path_out)
df.to_csv(path_data_out, index=False, encoding='utf-8', na_rep='')